# Analyse Statistique des Diplômés de l'Enseignement Supérieur Public Tunisien

**Projet :** Comprendre la production de diplômés en Tunisie : analyse par domaines, spécialités et établissements

**Étudiant :** Abderrahmen Youssef — Groupe DS4

---

## Problématique

Comment se répartissent les diplômés de l'enseignement supérieur public en Tunisie selon les domaines d'étude, les établissements, les universités et le genre, et quelles structures ou déséquilibres peut-on mettre en évidence à partir des données publiques disponibles ?

---

## Objectifs du projet

1. **Construire un jeu de données propre et lisible** à partir de la table `fact_diplomes` et des tables de correspondance des codes (universités, établissements, domaines, diplômes)

2. **Étudier la répartition des diplômés par sexe** en fonction des domaines, des spécialités et des établissements

3. **Mettre en évidence les domaines et groupes de formation les plus « productifs »** en termes de nombre de diplômés

4. **Comparer les profils de formation** des universités et des établissements (spécialisation, diversité de l'offre de formation, etc.)

5. **Analyser la structure globale du système de formation** à l'aide d'outils d'analyse statistique (analyses croisées et analyse multivariée)

---

## Architecture des données

Le projet repose sur un **modèle en étoile** :
- Une table principale (`fact_diplomes`) contenant les mesures quantitatives
- Quatre tables de correspondance permettant d'enrichir les codes :
  - `uni_code` : universités
  - `etablissment_code` : établissements
  - `domaines_code` : domaines d'étude
  - `diplomes_code` : types de diplômes

---

## 1. Préparation de l'environnement

### 1.1 Chargement des bibliothèques nécessaires

Nous allons utiliser les packages suivants :
- `readxl` : pour lire les fichiers Excel
- `dplyr` : pour la manipulation de données
- `tidyr` : pour le nettoyage des données
- `ggplot2` : pour la visualisation

In [17]:
# Installation des packages si nécessaire
# install.packages(c("readxl", "dplyr", "tidyr", "ggplot2"))

# Chargement des bibliothèques
library(readxl)
library(dplyr)
library(tidyr)
library(ggplot2)

---

## 2. Importation des données brutes

### 2.1 Définition des chemins

Structure du projet :
```
dataset/
├── raw/                  # Données brutes
│   ├── fact_diplomes.xls
│   ├── uni_code.xls
│   ├── etablissment_code.xls
│   ├── domaines_code.xls
│   └── diplomes_code.xls
└── merged/               # Données fusionnées (résultat)
```

In [18]:
# Définir le répertoire de travail
setwd("/home/abdou/Desktop/Higher-Education-Graduates-Analysis")

# Vérifier le répertoire actuel
getwd()

[1] "/home/abdou/Desktop/Higher-Education-Graduates-Analysis"

### 2.2 Chargement de la table principale (fact_diplomes)

Cette table contient les données quantitatives :
- `universite_code` : code de l'université
- `etablissement_code` : code de l'établissement
- `domain_code` : code du domaine d'étude
- `diplome_code` : code du type de diplôme
- `speciality` : spécialité de formation
- `group` : groupe ou famille de spécialités
- `diplomes_M` : nombre de diplômés hommes
- `diplomes_F` : nombre de diplômées femmes
- `diplomes_total` : nombre total de diplômés

In [19]:
# Charger la table des faits (fact_diplomes)
fact_diplomes <- read_excel("dataset/raw/fact_diplomes.xls")

# Afficher un aperçu des données
cat("\n=== TABLE PRINCIPALE : fact_diplomes ===")
cat("\nDimensions:", nrow(fact_diplomes), "lignes x", ncol(fact_diplomes), "colonnes\n")
print(head(fact_diplomes))
cat("\nStructure des données:\n")
str(fact_diplomes)


=== TABLE PRINCIPALE : fact_diplomes ===
Dimensions: 2083 lignes x 9 colonnes
# A tibble: 6 × 9
  universite_code etablissement_code domain_code diplome_code speciality   group
            <dbl>              <dbl>       <dbl> <chr>        <chr>        <chr>
1              13               1301          31 3            إدارة الأعم… تصرف 
2              13               1301          42 2            الأنظمة الم… إعلا…
3              13               1301          42 3            الأنظمة الم… إعلا…
4              13               1301          31 3            التصرف الصن… تصرف 
5              13               1301          31 11           التصرف المن… تصرف 
6              13               1301          53 2            التصرف في ا… هندس…
# ℹ 3 more variables: diplomes_M <dbl>, diplomes_F <dbl>, diplomes_total <dbl>

Structure des données:
tibble [2,083 × 9] (S3: tbl_df/tbl/data.frame)
 $ universite_code   : num [1:2083] 13 13 13 13 13 13 13 13 13 13 ...
 $ etablissement_code: num [1:2083]

### 2.3 Chargement des tables de référence (dimensions)

Ces tables permettent de décoder les codes présents dans la table principale :
1. **uni_code** : Codes et noms des universités
2. **etablissment_code** : Codes et noms des établissements
3. **domaines_code** : Codes et libellés des domaines d'étude
4. **diplomes_code** : Codes et types de diplômes

In [20]:
# Charger les tables de correspondance
uni_code <- read_excel("dataset/raw/uni_code.xls")
etablissment_code <- read_excel("dataset/raw/etablissment_code.xls")
domaines_code <- read_excel("dataset/raw/domaines_code.xls")
diplomes_code <- read_excel("dataset/raw/diplomes_code.xls")

# Afficher la STRUCTURE de chaque table (noms de colonnes)
cat("\n=== TABLE : Universités (uni_code) ===")
cat("\nNombre d'universités:", nrow(uni_code))
cat("\nNoms des colonnes:", paste(names(uni_code), collapse = ", "), "\n")
print(head(uni_code))

cat("\n=== TABLE : Établissements (etablissment_code) ===")
cat("\nNombre d'établissements:", nrow(etablissment_code))
cat("\nNoms des colonnes:", paste(names(etablissment_code), collapse = ", "), "\n")
print(head(etablissment_code))

cat("\n=== TABLE : Domaines (domaines_code) ===")
cat("\nNombre de domaines:", nrow(domaines_code))
cat("\nNoms des colonnes:", paste(names(domaines_code), collapse = ", "), "\n")
print(head(domaines_code))

cat("\n=== TABLE : Diplômes (diplomes_code) ===")
cat("\nNombre de types de diplômes:", nrow(diplomes_code))
cat("\nNoms des colonnes:", paste(names(diplomes_code), collapse = ", "), "\n")
print(head(diplomes_code))


=== TABLE : Universités (uni_code) ===
Nombre d'universités: 14
Noms des colonnes: code, university_fr, university_ar, siteweb, adresse, fax, tel 
# A tibble: 6 × 7
   code university_fr             university_ar     siteweb  adresse fax   tel  
  <dbl> <chr>                     <chr>             <chr>    <chr>   <chr> <chr>
1     1 UNIVERSITE EZZITOUNA      جامعة الزيتونة    www.uz.… 21, ru… 7157… 7157…
2     2 UNIVERSITE DE TUNIS       جامعة تونس        www.utu… 92, Av… 7156… 7156…
3     3 UNIVERSITE TUNIS EL MANAR جامعة تونس المنار www.utm… Campus… 7187… 7187…
4     4 UNIVERSITE DE CARTHAGE    جامعة  قرطاج      www.uca… Avenue… 7174… 71 7…
5     5 UNIVERSITE DE LA  MANOUBA جامعة منوبة       www.uma… Campus… 7160… 71 6…
6     6 UNIVERSITE DE JENDOUBA    جامعة جندوبة      www.uj.… Avenue… 7861… 78 6…

=== TABLE : Établissements (etablissment_code) ===
Nombre d'établissements: 206
Noms des colonnes: etablissement_code, university_code, label_ar, label_fr, website, gouvernorat, type, l

### 2.4 Inspection des noms de colonnes

Avant de fusionner, vérifions les noms exacts des colonnes dans toutes les tables.

In [21]:
# Afficher tous les noms de colonnes pour identification
cat("\n=== RÉCAPITULATIF DES COLONNES ===\n")
cat("\nfact_diplomes:", paste(names(fact_diplomes), collapse = ", "))
cat("\nuni_code:", paste(names(uni_code), collapse = ", "))
cat("\netablissment_code:", paste(names(etablissment_code), collapse = ", "))
cat("\ndomaines_code:", paste(names(domaines_code), collapse = ", "))
cat("\ndiplomes_code:", paste(names(diplomes_code), collapse = ", "), "\n")


=== RÉCAPITULATIF DES COLONNES ===

fact_diplomes: universite_code, etablissement_code, domain_code, diplome_code, speciality, group, diplomes_M, diplomes_F, diplomes_total
uni_code: code, university_fr, university_ar, siteweb, adresse, fax, tel
etablissment_code: etablissement_code, university_code, label_ar, label_fr, website, gouvernorat, type, lat, lon
domaines_code: domain_code, domaine_ar, domaine_fr
diplomes_code: diplome_code, label_fr, label_ar 


---

## 3. Fusion des données (Merge)

### 3.1 Stratégie de fusion

Nous allons fusionner les tables de référence avec la table principale `fact_diplomes` en utilisant des jointures (left join) pour conserver toutes les lignes de la table des faits.

**Ordre des jointures :**
1. Fusionner avec `uni_code` sur `universite_code`
2. Fusionner avec `etablissment_code` sur `etablissement_code`
3. Fusionner avec `domaines_code` sur `domain_code`
4. Fusionner avec `diplomes_code` sur `diplome_code`

### 3.2 Fusion avec les universités

In [24]:
# Étape 1 : Fusion avec la table des universités
# uni_code : code, university_fr (utiliser university_fr comme nom)
data_merged <- fact_diplomes %>%
  left_join(uni_code %>% select(code, university_fr), 
            by = c("universite_code" = "code")) %>%
  rename(universite_nom = university_fr)

cat("\nAprès fusion avec uni_code:")
cat("\nDimensions:", nrow(data_merged), "lignes x", ncol(data_merged), "colonnes")
cat("\nColonnes:", paste(names(data_merged), collapse = ", "), "\n")
print(head(data_merged, 3))


Après fusion avec uni_code:
Dimensions: 2083 lignes x 10 colonnes
Colonnes: universite_code, etablissement_code, domain_code, diplome_code, speciality, group, diplomes_M, diplomes_F, diplomes_total, universite_nom 
# A tibble: 3 × 10
  universite_code etablissement_code domain_code diplome_code speciality   group
            <dbl>              <dbl>       <dbl> <chr>        <chr>        <chr>
1              13               1301          31 3            إدارة الأعم… تصرف 
2              13               1301          42 2            الأنظمة الم… إعلا…
3              13               1301          42 3            الأنظمة الم… إعلا…
# ℹ 4 more variables: diplomes_M <dbl>, diplomes_F <dbl>, diplomes_total <dbl>,
#   universite_nom <chr>


### 3.3 Fusion avec les établissements

In [25]:
# Étape 2 : Fusion avec la table des établissements
# etablissment_code : etablissement_code, label_fr
# Convertir le code en caractère pour éviter les problèmes de type
data_merged <- data_merged %>%
  mutate(etablissement_code = as.character(etablissement_code)) %>%
  left_join(etablissment_code %>% select(etablissement_code, label_fr), 
            by = "etablissement_code") %>%
  rename(etablissement_nom = label_fr)

cat("\nAprès fusion avec etablissment_code:")
cat("\nDimensions:", nrow(data_merged), "lignes x", ncol(data_merged), "colonnes")
cat("\nColonnes:", paste(names(data_merged), collapse = ", "), "\n")
print(head(data_merged, 3))


Après fusion avec etablissment_code:
Dimensions: 2083 lignes x 11 colonnes
Colonnes: universite_code, etablissement_code, domain_code, diplome_code, speciality, group, diplomes_M, diplomes_F, diplomes_total, universite_nom, etablissement_nom 
# A tibble: 3 × 11
  universite_code etablissement_code domain_code diplome_code speciality   group
            <dbl> <chr>                    <dbl> <chr>        <chr>        <chr>
1              13 1301                        31 3            إدارة الأعم… تصرف 
2              13 1301                        42 2            الأنظمة الم… إعلا…
3              13 1301                        42 3            الأنظمة الم… إعلا…
# ℹ 5 more variables: diplomes_M <dbl>, diplomes_F <dbl>, diplomes_total <dbl>,
#   universite_nom <chr>, etablissement_nom <chr>


### 3.4 Fusion avec les domaines d'étude

In [26]:
# Étape 3 : Fusion avec la table des domaines
# domaines_code : domain_code, domaine_fr
data_merged <- data_merged %>%
  left_join(domaines_code %>% select(domain_code, domaine_fr), 
            by = "domain_code") %>%
  rename(domaine_nom = domaine_fr)

cat("\nAprès fusion avec domaines_code:")
cat("\nDimensions:", nrow(data_merged), "lignes x", ncol(data_merged), "colonnes")
cat("\nColonnes:", paste(names(data_merged), collapse = ", "), "\n")
print(head(data_merged, 3))


Après fusion avec domaines_code:
Dimensions: 2083 lignes x 12 colonnes
Colonnes: universite_code, etablissement_code, domain_code, diplome_code, speciality, group, diplomes_M, diplomes_F, diplomes_total, universite_nom, etablissement_nom, domaine_nom 
# A tibble: 3 × 12
  universite_code etablissement_code domain_code diplome_code speciality   group
            <dbl> <chr>                    <dbl> <chr>        <chr>        <chr>
1              13 1301                        31 3            إدارة الأعم… تصرف 
2              13 1301                        42 2            الأنظمة الم… إعلا…
3              13 1301                        42 3            الأنظمة الم… إعلا…
# ℹ 6 more variables: diplomes_M <dbl>, diplomes_F <dbl>, diplomes_total <dbl>,
#   universite_nom <chr>, etablissement_nom <chr>, domaine_nom <chr>


### 3.5 Fusion avec les types de diplômes

In [27]:
# Étape 4 : Fusion avec la table des diplômes
# diplomes_code : diplome_code, label_fr
data_merged <- data_merged %>%
  left_join(diplomes_code %>% select(diplome_code, label_fr), 
            by = "diplome_code") %>%
  rename(diplome_nom = label_fr)

cat("\nAprès fusion avec diplomes_code:")
cat("\nDimensions:", nrow(data_merged), "lignes x", ncol(data_merged), "colonnes")
cat("\nColonnes:", paste(names(data_merged), collapse = ", "), "\n")
print(head(data_merged, 3))


Après fusion avec diplomes_code:
Dimensions: 2083 lignes x 13 colonnes
Colonnes: universite_code, etablissement_code, domain_code, diplome_code, speciality, group, diplomes_M, diplomes_F, diplomes_total, universite_nom, etablissement_nom, domaine_nom, diplome_nom 
# A tibble: 3 × 13
  universite_code etablissement_code domain_code diplome_code speciality   group
            <dbl> <chr>                    <dbl> <chr>        <chr>        <chr>
1              13 1301                        31 3            إدارة الأعم… تصرف 
2              13 1301                        42 2            الأنظمة الم… إعلا…
3              13 1301                        42 3            الأنظمة الم… إعلا…
# ℹ 7 more variables: diplomes_M <dbl>, diplomes_F <dbl>, diplomes_total <dbl>,
#   universite_nom <chr>, etablissement_nom <chr>, domaine_nom <chr>,
#   diplome_nom <chr>


---

## 4. Nettoyage et réorganisation du dataset final

### 4.1 Sélection et réorganisation des colonnes

Nous allons créer un dataset final avec les colonnes dans un ordre logique et lisible.

In [32]:
# Réorganiser les colonnes - SANS les codes, uniquement les noms
diplomes_final <- data_merged %>%
  select(
    # Noms uniquement (pas de codes)
    universite_nom,
    etablissement_nom,
    domaine_nom,
    speciality,
    group,
    diplome_nom,
    
    # Statistiques sur les diplômés
    diplomes_M,
    diplomes_F,
    diplomes_total
  )

cat("\n=== DATASET FINAL FUSIONNÉ ===")
cat("\nDimensions:", nrow(diplomes_final), "lignes x", ncol(diplomes_final), "colonnes\n")
cat("\nAperçu des premières lignes:\n")
print(head(diplomes_final, 10))


=== DATASET FINAL FUSIONNÉ ===
Dimensions: 2083 lignes x 9 colonnes

Aperçu des premières lignes:
# A tibble: 10 × 9
   universite_nom     etablissement_nom domaine_nom speciality group diplome_nom
   <chr>              <chr>             <chr>       <chr>      <chr> <chr>      
 1 DIRECTION GENERAL… Institut Supérie… Affaires c… إدارة الأ… تصرف  Diplôme Na…
 2 DIRECTION GENERAL… Institut Supérie… Sciences  … الأنظمة ا… إعلا… Licence ap…
 3 DIRECTION GENERAL… Institut Supérie… Sciences  … الأنظمة ا… إعلا… Diplôme Na…
 4 DIRECTION GENERAL… Institut Supérie… Affaires c… التصرف ال… تصرف  Diplôme Na…
 5 DIRECTION GENERAL… Institut Supérie… Affaires c… التصرف ال… تصرف  Mastère pr…
 6 DIRECTION GENERAL… Institut Supérie… Ingénierie… التصرف في… هندس… Licence ap…
 7 DIRECTION GENERAL… Institut Supérie… Affaires c… التصرف في… تصرف  Licence ap…
 8 DIRECTION GENERAL… Institut Supérie… Sciences  … الشبكات و… إعلا… Diplôme Na…
 9 DIRECTION GENERAL… Institut Supérie… Services d… الوجستيك … تصرف… Lic

### 4.3 Statistiques descriptives du dataset fusionné

In [33]:
cat("\n=== STATISTIQUES DESCRIPTIVES ===")

# Nombre total de diplômés
total_diplomes <- sum(diplomes_final$diplomes_total, na.rm = TRUE)
total_hommes <- sum(diplomes_final$diplomes_M, na.rm = TRUE)
total_femmes <- sum(diplomes_final$diplomes_F, na.rm = TRUE)

cat("\nTotal général des diplômés:", total_diplomes)
cat("\n  - Hommes:", total_hommes, sprintf("(%.1f%%)", (total_hommes/total_diplomes)*100))
cat("\n  - Femmes:", total_femmes, sprintf("(%.1f%%)", (total_femmes/total_diplomes)*100))

# Nombre d'universités, établissements, domaines (utiliser les noms maintenant)
cat("\n\nNombre d'entités distinctes:")
cat("\n  - Universités:", n_distinct(diplomes_final$universite_nom))
cat("\n  - Établissements:", n_distinct(diplomes_final$etablissement_nom))
cat("\n  - Domaines:", n_distinct(diplomes_final$domaine_nom))
cat("\n  - Types de diplômes:", n_distinct(diplomes_final$diplome_nom))
cat("\n  - Spécialités:", n_distinct(diplomes_final$speciality))
cat("\n  - Groupes:", n_distinct(diplomes_final$group))


=== STATISTIQUES DESCRIPTIVES ===
Total général des diplômés: 55670
  - Hommes: 16722 (30.0%)
  - Femmes: 38948 (70.0%)

Nombre d'entités distinctes:
  - Universités: 14
  - Établissements: 189
  - Domaines: 22
  - Types de diplômes: 14
  - Spécialités: 1202
  - Groupes: 79

---

## 5. Export du dataset fusionné

In [34]:
# Créer le dossier merged s'il n'existe pas
if(!dir.exists("dataset/merged")) {
  dir.create("dataset/merged", recursive = TRUE)
  cat("Dossier 'dataset/merged' créé\n")
}

# Sauvegarder le dataset fusionné au format CSV
write.csv(diplomes_final, "dataset/merged/diplomes_fusionnes.csv", row.names = FALSE, fileEncoding = "UTF-8")

cat("\n✓ Dataset fusionné sauvegardé avec succès !")
cat("\n  Fichier: dataset/merged/diplomes_fusionnes.csv")
cat("\n  Dimensions:", nrow(diplomes_final), "lignes x", ncol(diplomes_final), "colonnes\n")


✓ Dataset fusionné sauvegardé avec succès !
  Fichier: dataset/merged/diplomes_fusionnes.csv
  Dimensions: 2083 lignes x 9 colonnes


---

## 6. Conclusion de l'étape de fusion

✅ **Étape 1 terminée : Fusion des données**

Nous avons réussi à :
1. Importer les 5 fichiers Excel (1 table de faits + 4 tables de référence)
2. Fusionner les tables en utilisant des jointures sur les codes
3. Créer un dataset enrichi avec les libellés explicites
4. Générer des statistiques descriptives complètes
5. Exporter le résultat dans `dataset/merged/diplomes_fusionnes.csv`

**Prochaines étapes :**
- Analyse descriptive approfondie
- Visualisations statistiques (répartition par genre, domaine, université)
- Analyses croisées par genre, domaine, université
- Analyse multivariée (ACP, clustering)

---